In [ ]:
import pandas as pd
import numpy as np
import numpy.ma as npma
import scipy as sc
from scipy import stats
from scipy.stats import norm, skew #for some statistics
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.kernel_ridge import KernelRidge
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from scipy.special import boxcox1p

from sklearn.linear_model import ElasticNet, Lasso, LassoLars,TheilSenRegressor, OrthogonalMatchingPursuit,  BayesianRidge, LassoLarsIC, Ridge,ARDRegression,SGDRegressor,HuberRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
os.sys.path.append(os.path.pardir)
from src.models.ConvexRegressor import ConvexRegressor
from src.models.ConvexRegressor import convregparam
from src.models.ConvexRegressor import convregconv

#############################################################################################################
project_dir =os.path.pardir
raw_path = os.path.join(project_dir,'data','processed')
train_path = os.path.join(raw_path, 'train.csv')
test_path = os.path.join(raw_path, 'test.csv')
sns.set()
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
train = train.drop(['Id','Unnamed: 0'],axis = 1)
test = test.drop(['Unnamed: 0'],axis = 1)
y = train['SalePrice'].values
Ids = test['Id'].values
train = train.drop('SalePrice',axis = 1)
flds = train.columns
train_x = train.values
test_x = test.drop('Id',axis = 1).values

#################################################################################################################
def Score(x, y, scaler, solver, q = 0.4, fldmask = None):
    scaler = scaler.fit(x)
    x = scaler.transform(x)
    yscaler = QuantileTransformer(output_distribution='normal').fit(y.reshape([-1,1]))
    y = yscaler.transform(y.reshape([-1,1])).ravel()
    if fldmask is not None:
        x = x[:,fldmask]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=q, random_state=0)   
    yscaler = QuantileTransformer(output_distribution='normal').fit(y_train.reshape([-1,1]))
    y_train = yscaler.transform(y_train.reshape([-1,1])).ravel()        
    solver = solver.fit(x_train, y_train)
    score = solver.score(x_test, y_test)
    print(score)
    return solver

def LearnAndPredict(x_test,x, y, scaler, solver, ids,issplit = True, plot = False, fldmask = None, filename = 'temp.csv'):
    yscaler = QuantileTransformer(output_distribution='normal').fit(y.reshape([-1,1]))
    y = yscaler.transform(y.reshape([-1,1])).ravel()
    if fldmask is not None:
        x = x[:,fldmask]
        x_test = x_test[:,fldmask]
    if scaler is not None:
        if issplit == False:
            rx = np.arange(x.shape[0])
            x = np.vstack((x,x_test))   
        scaler = scaler.fit(x)
        x = scaler.transform(x)
        if issplit == False:
            x_test = x[np.arange(rx.shape[0],x.shape[0])]
            x = x[rx]
        else:
            x_test = scaler.transform(x_test)
    solver = solver.fit(x, y)
    y = solver.predict(x_test)
    y = yscaler.inverse_transform(y.reshape([-1,1])).ravel()
    df_submission = pd.DataFrame({'Id': ids, 'SalePrice' : y} )
    submission_data_path = os.path.join(os.path.pardir,'data','external')
    submission_file_path = os.path.join(submission_data_path, filename)
    df_submission.to_csv(submission_file_path, index=False)
    if hasattr(solver, 'best_score_'):
        print(solver.best_score_)
    if hasattr(solver, 'best_params_'):
        print(solver.best_params_)
    if plot == True:
        sns.distplot(y,fit = norm)
    return solver

def linear_ml_solve(x, y, scaler, solver, q = 0.4):
    yscaler = QuantileTransformer(output_distribution='normal').fit(y.reshape([-1,1]))
    y = yscaler.transform(y.reshape([-1,1])).ravel()
    if scaler is not None:
        x = scaler.fit_transform(x)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=q, random_state=0)
    solver = solver.fit(x_train, y_train)
    score = solver.score(x_test,y_test)
    print(score)
    y = solver.predict(x_test)
    y = yscaler.inverse_transform(y.reshape([-1,1])).ravel()
    if hasattr(solver, 'best_score_'):
        print(solver.best_score_)
    if hasattr(solver, 'best_params_'):
        print(solver.best_params_)
    return y

# Some Global Parameters

In [ ]:
solver = LearnAndPredict(test_x,train_x, y, scaler = RobustScaler(), solver = Lasso(alpha = 0.001,max_iter = 10000,tol = 0.0001), ids = Ids, filename = 'LassoRidge.csv')
fldmask = solver.coef_ == 0.0

# 1. Lasso

In [ ]:
scaler = RobustScaler()

In [ ]:
solver = Lasso(alpha = 0.001,max_iter = 10000,tol = 0.0001)
Score(train_x, y, scaler, solver, q = 0.4, fldmask = fldmask)
#0.6838057972470732
Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)
#0.9182196591388715

In [ ]:

LearnAndPredict(test_x,train_x, y, scaler, solver = solver, ids = Ids, filename = 'Lasso.csv')

# 2.ARDRegression

In [ ]:
scaler = RobustScaler()

In [ ]:
solver = ARDRegression()
Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)
#0.9091333855168815

In [ ]:
LearnAndPredict(test_x,train_x, y, scaler, solver = solver, ids = Ids, fldmask =, filename = 'ARDRegression.csv')
#0.13142

In [ ]:
LearnAndPredict(test_x,train_x, y, scaler, solver = solver, ids = Ids, fldmask = fldmask, filename = 'ARDRegressionReduced.csv')
#0.24144

# 3.Ridge

In [ ]:
scaler = RobustScaler()
est1 = Ridge()
est2 = Lasso()
estimators = [est1,est2]
cest = ConvexRegressor(ests=estimators)
param_grid = convregparam(est2,{'alpha': [ 7.0, 6.0, 3.0, 0.02,0.001, 0.00099, 0.009, 0.0008]})
param_grid.update(convregparam(est1,{'alpha' : [7.0, 6.0, 4.0, 2.0, 0.6,0.5,0.55,0.45]}))
param_grid.update({'convparams' : [convregconv(estimators,[0.11, 0.89]), convregconv(estimators,[0.1,0.9]),
                                   convregconv(estimators,[0.09, 0.91]), convregconv(estimators,[0.05,0.95]),
                                   convregconv(estimators,[0.01,0.99]), convregconv(estimators,[0.001,0.999])]})

gsolver = GridSearchCV(cest, cv=5,param_grid=param_grid)
house_price = LearnAndPredict(test_x,train_x, y, scaler, solver = gsolver, ids = Ids, q = 0.4, filename = 'LassoRidge.csv')
#0.919300261382389
#{'Lasso$$&&$$alpha': 0.001, 'Ridge$$&&$$alpha': 7.0, 'convparams': {'Ridge': 0.001, 'Lasso': 0.999}}

# 4. SGDRegressor

In [ ]:
scaler = RobustScaler()
loss = ['squared_loss','huber','epsilon_insensitive','squared_epsilon_insensitive']
penalty = ['l1','l2']
param_grid = { 'alpha': [0.0005, 0.0009, 0.001, 0.0015, 0.0019, 0.002,0.005,0.01,0.05,0.1]}
#Such as this task is solved by iteration so the is no reason to use gread search routines.

  ## 4.1 Huber + L1

In [ ]:
solver = SGDRegressor(alpha = 0.001, loss = 'huber', penalty = 'l1', epsilon = 1.5, max_iter = 200000, warm_start = True)

In [ ]:
#solver = SGDRegressor(alpha = 0.001, loss = 'huber', penalty = 'l1', epsilon = 0.1, max_iter = 150000, warm_start = True)
solver = LearnAndPredict(test_x,train_x, y, scaler, solver = solver, ids = Ids, filename = 'SGDRegressor_huber_l1.csv')
#0.124

In [ ]:
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)
#0.9184762393457621

## 4.2 Epsilon_insensitive + L1

In [ ]:
solver = SGDRegressor(alpha = 0.001, loss = 'epsilon_insensitive',epsilon = 0.1, penalty = 'l1', max_iter = 150000, warm_start = True)
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)
#0.855235126971956

In [ ]:
solver.max_iter = 300000
solver = LearnAndPredict(test_x,train_x, y, scaler, solver = solver, ids = Ids, filename = 'SGDRegressor_EpsilonInsensitive_l1.csv')
#0.13001

## 4.3 Squared_Epsilon_Insensitive + L1

In [ ]:
solver = SGDRegressor(alpha = 0.001, loss = 'squared_epsilon_insensitive',epsilon = 0.1, penalty = 'l1', max_iter = 150000, warm_start = True)

In [ ]:
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)

# 5 Huber Regressor

In [ ]:
scaler = RobustScaler()

In [ ]:
solver = HuberRegressor(epsilon=2.35, max_iter=1000, alpha=0.0001, warm_start=True, fit_intercept=True, tol=1e-05)

In [ ]:
solver = HuberRegressor(epsilon=2.35, max_iter=1000, alpha=0.0001, warm_start=True, fit_intercept=True, tol=1e-05)
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)
#0.9213839425014468

In [ ]:
solver = LearnAndPredict(test_x,train_x, y, scaler, solver = solver, ids = Ids, filename = 'HuberRegressor.csv')

In [ ]:
solver = HuberRegressor(epsilon=2.35, max_iter=10000, alpha=0.0001, warm_start=False, fit_intercept=True, tol=1e-05)
param_grid = {'alpha': [0.00001, 0.00005,0.00008, 0.0001, 0.0005, 0.0009, 0.001, 0.0015, 0.0019, 0.002]}#,
             #'epsilon' : [1.0, 1.5, 2.0, 2.35, 2.5, 2.75,3.0]}
solver = GridSearchCV(solver, cv=5,param_grid=param_grid)
solver = LearnAndPredict(test_x,train_x, y, scaler, solver = solver, ids = Ids, filename = 'HuberRegressor.csv')

In [ ]:
solver = HuberRegressor(epsilon=2.35, max_iter=1000, alpha=0.0001, warm_start=True, fit_intercept=True, tol=1e-05)
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)

In [ ]:
solver = HuberRegressor(epsilon=2.35, max_iter=1000, alpha=0.0001, warm_start=True, fit_intercept=True, tol=1e-05)
param_grid = {'alpha': [0.00001, 0.00005,0.00008, 0.0001, 0.0005, 0.0009, 0.001, 0.0015, 0.0019, 0.002],
             'epsilon' : [2.7, 2.71, 2.72, 2.73, 2.74, 2.75, 2.76, 2.77, 2.78, 2.79, 2.8]}
solver = GridSearchCV(solver, cv=5,param_grid=param_grid)
solver = LearnAndPredict(test_x,train_x, y, scaler, solver = solver, ids = Ids, filename = 'HuberRegressor.csv')
#this the last one and it is the most high performance
#0.9158018800233361
#{'alpha': 5e-05, 'epsilon': 2.7}

In [ ]:
solver = HuberRegressor(epsilon=2.35, max_iter=1000, alpha=0.0001, warm_start=True, fit_intercept=True, tol=1e-05)
solver = LearnAndPredict(test_x,train_x, y, scaler, solver = solver, ids = Ids, issplit = False,  filename = 'HuberRegressor.csv')

 # 6. OrthogonalMatchingPursuit

In [ ]:
solver = OrthogonalMatchingPursuit(n_nonzero_coefs=100, tol=None, fit_intercept=False, normalize=True, precompute='auto')
scaler = StandardScaler()
scaler = RobustScaler()
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)

In [ ]:
solver = OrthogonalMatchingPursuit(n_nonzero_coefs=None, tol=None, fit_intercept=False, normalize=True, precompute='auto')
#scaler = StandardScaler()
scaler = RobustScaler()
solver = LearnAndPredict(test_x,train_x, y, scaler, solver = solver, ids = Ids,  filename = 'ORMATCHPURS.csv')
#Not so good method - only 0.125

# 7. TheilSenRegressor

In [ ]:
scaler = RobustScaler()
solver = TheilSenRegressor(fit_intercept=True, copy_X=True, max_subpopulation=10000.0, 
                           n_subsamples=None, max_iter=300, tol=0.001, random_state=None, n_jobs=None, verbose=False)
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)

In [ ]:
scaler = RobustScaler()
solver = TheilSenRegressor(fit_intercept=True, copy_X=True, max_subpopulation=10000.0, 
                           n_subsamples=None, max_iter=3000, tol=0.001, random_state=None, n_jobs=None, verbose=False)
solver = LearnAndPredict(test_x,train_x, y, scaler, solver = solver, ids = Ids,  filename = 'TheilSenReg.csv')

# 8. BayesianRidge

In [ ]:
scaler = RobustScaler()
solver = BayesianRidge(n_iter=1000, tol=0.001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06, 
                       compute_score=False, fit_intercept=True, normalize=False, copy_X=True, verbose=False)
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)

In [ ]:
scaler = RobustScaler()
solver = BayesianRidge(n_iter=30000, tol=0.001, alpha_1=1e-09, alpha_2=8e-01, lambda_1=8e-01, lambda_2=1e-09, 
                       compute_score=False, fit_intercept=True, normalize=False, copy_X=True, verbose=False)
param_grid = {'alpha_1': [1e-07, 1e-06,1e-05,1e-04,1e-03,1e-02,1e-01],
             'alpha_2' : [1e-01,1.3e-01,1.5e-01,2e-01, 2.5e-01, 3e-01],
             'lambda_2': [1e-07, 1e-06,1e-05,1e-04,1e-03,1e-02,1e-01],
             'lambda_1': [1e-01,1.3e-01,1.5e-01,2e-01, 2.5e-01, 3e-01]}
#solver = GridSearchCV(solver, cv=5,param_grid=param_grid)
solver = LearnAndPredict(test_x,train_x, y, scaler, solver = solver, ids = Ids,  filename = 'BayesianRidge.csv')
#0.9159600824373367
#{'alpha_1': 1e-07, 'alpha_2': 0.1, 'lambda_1': 0.1, 'lambda_2': 1e-07}
#0.12264 - for optimal given above
#0.12267 - for default implementation 
#0.9157352390453596
#{'alpha_1': 1e-07, 'alpha_2': 0.3, 'lambda_1': 0.1, 'lambda_2': 0.1}
#0.12361
#0.9159698654626581
#{'alpha_1': 1e-07, 'alpha_2': 0.3, 'lambda_1': 0.3, 'lambda_2': 1e-07}
#0.12259
#0.12254
#0.12246

In [ ]:
scaler = RobustScaler()
solver = BayesianRidge(n_iter=1000, tol=0.001, alpha_1=1e-07, alpha_2=1e-01, lambda_1=1e-06, lambda_2=1e-06, 
                       compute_score=False, fit_intercept=True, normalize=False, copy_X=True, verbose=False)

# 9. Convex combination

In [ ]:
scaler = RobustScaler()
s1 = HuberRegressor(epsilon=2.7, max_iter=1000, alpha=5e-05, warm_start=False, fit_intercept=False, tol=1e-05)
s2 = Lasso(alpha = 0.001,max_iter = 10000,tol = 0.0001)
s3 = BayesianRidge(n_iter=1000, tol=0.001, alpha_1=1e-08, alpha_2=0.5, lambda_1=0.5, lambda_2=1e-08, 
                       compute_score=False, fit_intercept=True, normalize=False, copy_X=True, verbose=False)
estimators = [s1,s2,s3]
solver = ConvexRegressor(ests=estimators,convparams = convregconv(estimators,[0.3,0,0.8]))
param_grid1 = {'convparams' : [convregconv(estimators,[0.5, 0.4,0.1]), convregconv(estimators,[0.6,0.3,0.1]),
                                convregconv(estimators,[0.6, 0.1,0.3]), convregconv(estimators,[0.6,0.0,0.4]),
                                convregconv(estimators,[0.6,0.2,0.2]), convregconv(estimators,[0.8,0.1,0.1])]}

param_grid2 = {'convparams' : [convregconv(estimators,[0.5, 0.25,0.0]), convregconv(estimators,[0.6,0.4,0]),
                                convregconv(estimators,[0.7, 0.3,0]), convregconv(estimators,[0.8,0.2,0]),
                                convregconv(estimators,[0.9,0.1,0]), convregconv(estimators,[0.4,0.6,0])]}

#solver = GridSearchCV(solver, cv=5,param_grid=param_grid1)
solver = LearnAndPredict(test_x,train_x, y, scaler = scaler, solver = solver, ids = Ids,  filename = 'HuberLasso.csv')
#0.9187263622916003
#{'convparams': {'HuberRegressor': 0.4, 'Lasso': 0.6, 'BayesianRidge': 0}}
#0.12248
#0.9179347873508138
#{'convparams': {'HuberRegressor': 0.5, 'Lasso': 0.4, 'BayesianRidge': 0.1}}
#0.12197
#convregconv(estimators,[0.7,0,0.3]) - 0.12196
#convregconv(estimators,[0.5,0,0.5]) - 0.12175
#convregconv(estimators,[0.3,0,0.8]) - 0.12775

# 10. Kernel Ridge

In [ ]:
scaler = RobustScaler()
solver = KernelRidge(alpha=35, kernel='linear', gamma=1, degree=1, coef0=1, kernel_params=None)
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)
#degree=3   0.9113532569817305
#0.9192918459212212
#0.9196149350223377 - alpha 20
#0.9197754264469626 - alpha 30

In [ ]:
scaler = RobustScaler()
solver = KernelRidge(alpha=25, kernel='linear', gamma=1, degree=1, coef0=1, kernel_params=None)
solver = LearnAndPredict(test_x,train_x, y, scaler = scaler, solver = solver, ids = Ids,  filename = 'KernelRidge.csv')
#0.12088 

# SVR

In [ ]:
scaler = RobustScaler()
solver = SVR(kernel='linear', degree=1, gamma=0.001, coef0=0.0, tol=0.001, C=1, epsilon=0.5, shrinking=True,
             cache_size=200, verbose=False,max_iter=-1)
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)

In [ ]:
scaler = RobustScaler()
solver = SVR(kernel='linear', degree=1, gamma=0.001, coef0=0.0, tol=0.001, C=1, epsilon=0.1, shrinking=True,
             cache_size=200, verbose=False,max_iter=-1)
solver = LearnAndPredict(test_x,train_x, y, scaler = scaler, solver = solver, ids = Ids,  filename = 'SVR.csv')

# LinearSVR

In [ ]:
scaler = RobustScaler()
solver = LinearSVR(epsilon=0.0, tol=0.0001, C=0.01, loss='epsilon_insensitive', fit_intercept=True, intercept_scaling=1.0, dual=True, verbose=0, random_state=None, max_iter=1000)
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)
#0.9183937382600859 C = 0.01

In [ ]:
scaler = RobustScaler()
solver = LinearSVR(epsilon=0.0, tol=0.0001, C=0.01, loss='epsilon_insensitive', fit_intercept=True,
                   intercept_scaling=1.0, dual=True, verbose=0, random_state=None, max_iter=1000)
param_grid = {'C' : [0.1, 0.05, 0.01, 0.009, 0.006, 0.003, 0.001, 0.0005, 0.0001]}

#solver = GridSearchCV(solver, cv=5,param_grid=param_grid)
solver = LearnAndPredict(test_x,train_x, y, scaler = scaler, solver = solver, ids = Ids,  filename = 'LinearSVR.csv')


# KNeighborsRegressor

In [ ]:
solver = KNeighborsRegressor(n_neighbors=5, weights='distance', algorithm='auto', leaf_size=30, p=2, metric='minkowski',
                    metric_params=None, n_jobs=None)
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)

# RadiusNeighborsRegressor

In [ ]:
solver = RadiusNeighborsRegressor(radius=1.0, weights='uniform', algorithm='auto',
                                  leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)
solver = Score(train_x, y, scaler, solver, q = 0.4, fldmask = None)